In [1]:
import pandas as pd
import numpy as np
from os.path import exists
import re

In [2]:
tf_idf = pd.read_table("./../MMSR_WT22_Task1_Data/id_lyrics_tf-idf_mmsr.tsv", index_col="id")
tf_idf.head()

,abl,accept,across,act,addict,afraid,age,ago,ah,ahead,...,yea,yeah,year,yellow,yes,yesterday,yet,yo,young,youth
id,,,,,,,,,,,,,,,,,,,,,
9jbSytob9XRzwvB6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.150511
Njp6JPM8vitbhVJU,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
h48f46ZsT9h0Z5Dm,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.149783,0.0,0.0,0.0,0.000000
ZmXVK43zlqdeq6z8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
PV5EXN6AIVBqvsLO,0.0,0.0,0.0,0.0,0.0,0.327025,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [3]:
# Other Feature vectors to use
word2vec = pd.read_table("./../MMSR_WT22_Task1_Data/id_lyrics_word2vec_mmsr.tsv", index_col='id')
bert = pd.read_table("./../MMSR_WT22_Task1_Data/id_bert_mmsr.tsv", index_col='id')

In [4]:
genres = pd.read_table("./../MMSR_WT22_Task1_Data/id_genres_mmsr.tsv", index_col="id")

genres.head()

,genre
id,
0009fFIM1eYThaPg,['pop']
0010xmHR6UICBOYT,"['beats', 'underground hip hop', 'lo fi']"
002Jyd0vN4HyCpqL,"['hard rock', 'classic rock', 'rock', 'progres..."
006TYKNjNxWjfKjy,"['power metal', 'symphonic metal', 'symphonic ..."
007LIJOPQ4Sb98qV,"['post punk', 'new wave', 'dream pop', 'altern..."


In [5]:
info = pd.read_table("./../MMSR_WT22_Task1_Data/id_information_mmsr.tsv", index_col="id")
info.head()

,artist,song,album_name
id,,,
0009fFIM1eYThaPg,Cheryl,Rain on Me,3 Words
0010xmHR6UICBOYT,Oddisee,After Thoughts,The Beauty in All
002Jyd0vN4HyCpqL,Blue Öyster Cult,ME 262,Secret Treaties
006TYKNjNxWjfKjy,Rhapsody,Flames of Revenge,Legendary Years (Re-Recorded)
007LIJOPQ4Sb98qV,The Chameleons,Nostalgia,What Does Anything Mean? Basically (2009 Remas...


In [6]:
# Check if there is any null value in tf_idf values
tf_idf.isnull().sum().sum()

0

In [7]:
def cosine_similarity(d1, d2):
    divisor = np.linalg.norm(d1) * np.linalg.norm(d2)
    if divisor == 0:
        return 0
    return (d1 @ d2) / divisor

In [8]:
def inner_product(d1, d2):
    return (d1 @ d2) 

In [9]:
# I am not sure if it is correct
def jaccard_formulation(d1, d2):
    divisor = np.linalg.norm(d1) + np.linalg.norm(d2) - (d1 @ d2)
    if divisor == 0:
        return 0
    return (d1 @ d2) / divisor

In [10]:
# Some calculations
print(
    cosine_similarity(tf_idf.iloc[0].values, tf_idf.iloc[2].values),
    cosine_similarity(tf_idf.iloc[0].values, tf_idf.iloc[3].values)
)

0.08598512652034522 0.12868134189383443


In [11]:
# Matrix to store the values
# cosine_sim = np.zeros((len(tf_idf.index),len(tf_idf.index)))

# cosine_sim = np.zeros((len(tf_idf.index),len(tf_idf.index)))
# print(cosine_sim.shape) # (76115, 76115) 

# To calculate the similarity between all songs
# for row, id1 in enumerate(tf_idf.index):
#     for col,id2 in enumerate(tf_idf.index):
#         # print(row,col)
#         cosine_sim[row,col] = cosine_similarity(tf_idf.loc[id1].values, tf_idf.loc[id2].values)

In [12]:
def getSongIdByQuery(query):
    artist, track =query.split(',')
    id_ = info[(info['artist'] == artist) & (info['song'] == track)].index.values[0]
    return id_

In [13]:
# The index depends in the features vector, so it is better to assign 
# it depending on which feature vector we are using

# change to bert.index or word2vec.index
index_values = tf_idf.index

if exists('cosine_distances_tfidf.csv'):
    df_cosineDistance_tfidf = pd.read_csv("cosine_distances_tfidf.csv", index_col="id")
else:
    df_cosineDistance_tfidf = pd.DataFrame(index=index_values)
    
if exists('innerProduct_distances_tfidf.csv'):
    df_innerProductDistance_tfidf = pd.read_csv("innerProduct_distances_tfidf.csv", index_col="id")
else:
    df_innerProductDistance_tfidf = pd.DataFrame(index=index_values)
    
if exists('jaccard_distances_tfidf.csv'):
    df_jaccardDistance_tfidf = pd.read_csv("jaccard_distances_tfidf.csv", index_col="id")
else:
    df_jaccardDistance_tfidf = pd.DataFrame(index=index_values)
    
    
index_values = word2vec.index

if exists('cosine_distances_word2vec.csv'):
    df_cosineDistance_word2vec = pd.read_csv("cosine_distances_word2vec.csv", index_col="id")
else:
    df_cosineDistance_word2vec = pd.DataFrame(index=index_values)
    
if exists('innerProduct_distances_word2vec.csv'):
    df_innerProductDistance_word2vec = pd.read_csv("innerProduct_distances_word2vec.csv", index_col="id")
else:
    df_innerProductDistance_word2vec = pd.DataFrame(index=index_values)
    
if exists('jaccard_distances_word2vec.csv'):
    df_jaccardDistance_word2vec = pd.read_csv("jaccard_distances_word2vec.csv", index_col="id")
else:
    df_jaccardDistance_word2vec = pd.DataFrame(index=index_values)
    
    
index_values = bert.index

if exists('cosine_distances_bert.csv'):
    df_cosineDistance_bert = pd.read_csv("cosine_distances_bert.csv", index_col="id")
else:
    df_cosineDistance_bert = pd.DataFrame(index=index_values)
    
if exists('innerProduct_distances_bert.csv'):
    df_innerProductDistance_bert = pd.read_csv("innerProduct_distances_bert.csv", index_col="id")
else:
    df_innerProductDistance_bert = pd.DataFrame(index=index_values)
    
if exists('jaccard_distances_bert.csv'):
    df_jaccardDistance_bert = pd.read_csv("jaccard_distances_bert.csv", index_col="id")
else:
    df_jaccardDistance_bert = pd.DataFrame(index=index_values)

In [14]:

df_cosineDistance_tfidf.head()

,007LIJOPQ4Sb98qV,0009fFIM1eYThaPg,wdAhzJrYFsHfCyCl
id,,,
9jbSytob9XRzwvB6,0.048452,0.006539,0.051474
Njp6JPM8vitbhVJU,0.001441,0.013563,0.009384
h48f46ZsT9h0Z5Dm,0.102718,0.015595,0.040152
ZmXVK43zlqdeq6z8,0.098409,0.015704,0.028695
PV5EXN6AIVBqvsLO,0.013196,0.017603,0.015591


In [15]:
def distanceToSongs(idSong, similarity_function, df, features_vector):
    if idSong in df.columns.values:
        print("Already in data")
    else:
        songs = features_vector.index.values
        distances = [similarity_function(features_vector.loc[idSong], features_vector.loc[song]) for index,song in enumerate(songs)]
        df[idSong]  = distances 

In [16]:
#df_cosineDistance['9jbSytob9XRzwvB6'] 

In [17]:
# User query "The Chameleons, Nostalgia"
info[(info['artist'] == 'The Chameleons') & (info['song'] == 'Nostalgia')]

,artist,song,album_name
id,,,
007LIJOPQ4Sb98qV,The Chameleons,Nostalgia,What Does Anything Mean? Basically (2009 Remas...


In [18]:
# Sample

# artist,song
id_song = getSongIdByQuery("The Chameleons,Nostalgia")
print(id_song)
distanceToSongs(id_song, cosine_similarity, df_cosineDistance_tfidf, tf_idf)

007LIJOPQ4Sb98qV
Already in data


In [19]:
df_cosineDistance_tfidf.head()

,007LIJOPQ4Sb98qV,0009fFIM1eYThaPg,wdAhzJrYFsHfCyCl
id,,,
9jbSytob9XRzwvB6,0.048452,0.006539,0.051474
Njp6JPM8vitbhVJU,0.001441,0.013563,0.009384
h48f46ZsT9h0Z5Dm,0.102718,0.015595,0.040152
ZmXVK43zlqdeq6z8,0.098409,0.015704,0.028695
PV5EXN6AIVBqvsLO,0.013196,0.017603,0.015591


In [20]:
distanceToSongs(getSongIdByQuery("Cheryl,Rain on Me"), cosine_similarity, df_cosineDistance_tfidf, tf_idf)
distanceToSongs(getSongIdByQuery("Doda,Riotka"), cosine_similarity, df_cosineDistance_tfidf, tf_idf)

Already in data
Already in data


In [21]:
# Default width of column and number of columns
print(pd.options.display.max_colwidth)
print(pd.options.display.max_columns)

50
20


In [22]:
# Set values to None to displau all the data
# pd.set_option("max_columns", None) #In some versions raises an error
# pd.set_option("max_colwidth", None)

pd.options.display.max_colwidth = None
pd.options.display.max_columns = None

In [23]:
def getTopValues(idSong, df_metricUsed):
    top_values = df_metricUsed[idSong].sort_values(ascending=False)
    return genres.loc[top_values.index].join(info, on="id", how="left")

In [24]:
getTopValues('007LIJOPQ4Sb98qV', df_cosineDistance_tfidf).head(11)

,genre,artist,song,album_name
id,,,,
007LIJOPQ4Sb98qV,"['post punk', 'new wave', 'dream pop', 'alternative rock', 'madchester', 'rock']",The Chameleons,Nostalgia,What Does Anything Mean? Basically (2009 Remaster)
fqotTYeTLpnRj3lF,"['pop', 'teen pop', 'rock', 'pop rock', 'disney', 'hip hop', 'grunge', 'alternative rock', 'rap', 'power pop', 'singer songwriter', 'synthpop']",Hilary Duff,So Yesterday,Metamorphosis
E7t3Y9cW86HLOFZp,"['electronica', 'electro', 'disco', 'funk', 'house', 'indietronica', 'dance rock', 'rock', 'electropop', 'indie rock', 'indie pop', 'electro house', 'space rock', 'pop', 'techno', 'beats', 'disco house', 'indie electronica', 'dance punk', 'background music']",Midnight Juggernauts,Shadows,Shadows
wdAhzJrYFsHfCyCl,"['electropop', 'pop', 'synthpop']",Doda,Riotka,Riotka
wbUm7Kdu4u7CSG39,"['disco', 'singer songwriter', 'classic soul']",Grace Jones,Tomorrow,Portfolio
8crboBOJbzovC9SC,['psychedelic rock'],Ulver,Where Is Yesterday,Childhood's End
gcFfKAX8vSRWOtqk,"['psychedelic rock', 'experimental', 'rock', 'art rock', 'classic rock', 'baroque', 'choral', 'baroque pop', 'psychedelic pop', 'gregorian chant']",The United States of America,Where Is Yesterday,The United States Of America
ExbzgBOxVe8jTGKx,"['pop', 'singer songwriter', 'rumba']",Colbie Caillat,Like Yesterday,All Of You
qSEy5OiW84jPqQb5,"['classic rock', 'rock', 'pop rock', 'pop', 'soft rock', 'hard rock', 'progressive rock', 'new wave', 'easy listening', 'metal', 'alternative rock', 'singer songwriter', 'album rock', 'glam metal']",Foreigner,That Was Yesterday,Agent Provocateur


In [25]:
# Distances with innerProduct
distanceToSongs(getSongIdByQuery("The Chameleons,Nostalgia"), inner_product, df_innerProductDistance_tfidf, tf_idf)
distanceToSongs(getSongIdByQuery("Cheryl,Rain on Me"), inner_product, df_innerProductDistance_tfidf, tf_idf)
distanceToSongs(getSongIdByQuery("Doda,Riotka"), inner_product, df_innerProductDistance_tfidf, tf_idf)

Already in data
Already in data
Already in data


In [26]:
# Distances with jaccard similarity
distanceToSongs(getSongIdByQuery("The Chameleons,Nostalgia"), jaccard_formulation, df_jaccardDistance_tfidf, tf_idf)
distanceToSongs(getSongIdByQuery("Cheryl,Rain on Me"), jaccard_formulation, df_jaccardDistance_tfidf, tf_idf)
distanceToSongs(getSongIdByQuery("Doda,Riotka"), jaccard_formulation, df_jaccardDistance_tfidf, tf_idf)

Already in data
Already in data
Already in data


In [27]:
getTopValues('0009fFIM1eYThaPg', df_innerProductDistance_tfidf).head(11)

,genre,artist,song,album_name
id,,,,
0009fFIM1eYThaPg,['pop'],Cheryl,Rain on Me,3 Words
Bp2C0upIsbgDIj6g,"['pop', 'rain', 'singer songwriter', 'water', 'riot grrrl']",Cyndi Lauper,Who Let in the Rain,Hat Full Of Stars
ao7dLWmpTymdh4NP,"['pop', 'britpop', 'rain', 'jazz', 'rock', 'electropop', 'soul', 'post rock', 'funk', 'acid jazz', 'alternative pop', 'electronica', 'alternative rock', 'power pop']",Mika,Rain,The Boy Who Knew Too Much
WJRG3U9F9ugC8EGH,['indie rock'],Efek Rumah Kaca,Hujan Jangan Marah,Kamar Gelap
AnOdZ5pDIL2LhtCb,"['quiet storm', 'soul', 'rain', 'r b', 'pop', 'hip hop', 'smooth soul', 'funk', 'jazz funk', 'new jack swing', 'rhythm and blues', 'girl group']",SWV,Rain,Release Some Tension
pxgFoLb5tSjKO0Rr,"['soul', 'pop', 'jazz', 'rock', 'singer songwriter', 'rain', 'lounge', 'ambient', 'indie pop', 'progressive rock', 'easy listening', 'britpop', 'alternative pop', 'neo soul', 'pop folk', 'drama']",Duffy,Rain on Your Parade,Rockferry
ARAbmSl33s9qdv3W,"['pop', 'rock', 'soul', 'funk', 'classic rock', 'soundtrack', 'singer songwriter', 'rain', 'rhythm and blues', 'pop rock', 'easy listening', 'soft rock', 'new wave']",Prince,Purple Rain,Purple Rain
cUtnKV2vkqB3hhkX,"['pop', 'rock', 'funk', 'soul', 'soundtrack', 'soft rock', 'classic rock', 'new wave', 'r b', 'pop rock', 'rain', 'gospel', 'jazz', 'hip hop', 'hard rock', 'singer songwriter', 'psychedelic soul', 'orchestra', 'sleep']",Prince & The Revolution,Purple Rain,Purple Rain
HFLuvJXc6SjcJt7d,"['pop', 'rain']",Cher,"Rain, Rain",I Got You Babe / It's Gonna Rain [Digital 45]


In [28]:
df_cosineDistance_tfidf.head()

,007LIJOPQ4Sb98qV,0009fFIM1eYThaPg,wdAhzJrYFsHfCyCl
id,,,
9jbSytob9XRzwvB6,0.048452,0.006539,0.051474
Njp6JPM8vitbhVJU,0.001441,0.013563,0.009384
h48f46ZsT9h0Z5Dm,0.102718,0.015595,0.040152
ZmXVK43zlqdeq6z8,0.098409,0.015704,0.028695
PV5EXN6AIVBqvsLO,0.013196,0.017603,0.015591


In [29]:
df_cosineDistance_tfidf.loc['007LIJOPQ4Sb98qV','007LIJOPQ4Sb98qV']

1.0000000000000002

In [30]:
df_jaccardDistance_tfidf.head()


,007LIJOPQ4Sb98qV,0009fFIM1eYThaPg,wdAhzJrYFsHfCyCl
id,,,
9jbSytob9XRzwvB6,0.024827,0.003280,0.026417
Njp6JPM8vitbhVJU,0.000721,0.006828,0.004714
h48f46ZsT9h0Z5Dm,0.054139,0.007859,0.020487
ZmXVK43zlqdeq6z8,0.051751,0.007914,0.014556
PV5EXN6AIVBqvsLO,0.006642,0.008880,0.007857


In [31]:
df_jaccardDistance_tfidf.loc['007LIJOPQ4Sb98qV','007LIJOPQ4Sb98qV']

1.0

## The distance is the same with the inner product and cosine similarities because vectors are normalized

In [32]:
distanceToSongs(getSongIdByQuery("The Chameleons,Nostalgia"), jaccard_formulation, df_jaccardDistance_word2vec, word2vec)
distanceToSongs(getSongIdByQuery("Cheryl,Rain on Me"), jaccard_formulation, df_jaccardDistance_word2vec, word2vec)
distanceToSongs(getSongIdByQuery("Doda,Riotka"), jaccard_formulation, df_jaccardDistance_word2vec, word2vec)

Already in data
Already in data
Already in data


In [33]:
distanceToSongs(getSongIdByQuery("The Chameleons,Nostalgia"), inner_product, df_innerProductDistance_bert, bert)
distanceToSongs(getSongIdByQuery("Cheryl,Rain on Me"), inner_product, df_innerProductDistance_bert, bert)
distanceToSongs(getSongIdByQuery("Doda,Riotka"), inner_product, df_innerProductDistance_bert, bert)

Already in data
Already in data
Already in data


# Comparison between CosineSimilarity with TF-IDF and Jaccard Formulation with word2vec

In [34]:
getTopValues('0009fFIM1eYThaPg', df_cosineDistance_tfidf).head(11)


,genre,artist,song,album_name
id,,,,
0009fFIM1eYThaPg,['pop'],Cheryl,Rain on Me,3 Words
Bp2C0upIsbgDIj6g,"['pop', 'rain', 'singer songwriter', 'water', 'riot grrrl']",Cyndi Lauper,Who Let in the Rain,Hat Full Of Stars
ao7dLWmpTymdh4NP,"['pop', 'britpop', 'rain', 'jazz', 'rock', 'electropop', 'soul', 'post rock', 'funk', 'acid jazz', 'alternative pop', 'electronica', 'alternative rock', 'power pop']",Mika,Rain,The Boy Who Knew Too Much
WJRG3U9F9ugC8EGH,['indie rock'],Efek Rumah Kaca,Hujan Jangan Marah,Kamar Gelap
AnOdZ5pDIL2LhtCb,"['quiet storm', 'soul', 'rain', 'r b', 'pop', 'hip hop', 'smooth soul', 'funk', 'jazz funk', 'new jack swing', 'rhythm and blues', 'girl group']",SWV,Rain,Release Some Tension
pxgFoLb5tSjKO0Rr,"['soul', 'pop', 'jazz', 'rock', 'singer songwriter', 'rain', 'lounge', 'ambient', 'indie pop', 'progressive rock', 'easy listening', 'britpop', 'alternative pop', 'neo soul', 'pop folk', 'drama']",Duffy,Rain on Your Parade,Rockferry
ARAbmSl33s9qdv3W,"['pop', 'rock', 'soul', 'funk', 'classic rock', 'soundtrack', 'singer songwriter', 'rain', 'rhythm and blues', 'pop rock', 'easy listening', 'soft rock', 'new wave']",Prince,Purple Rain,Purple Rain
cUtnKV2vkqB3hhkX,"['pop', 'rock', 'funk', 'soul', 'soundtrack', 'soft rock', 'classic rock', 'new wave', 'r b', 'pop rock', 'rain', 'gospel', 'jazz', 'hip hop', 'hard rock', 'singer songwriter', 'psychedelic soul', 'orchestra', 'sleep']",Prince & The Revolution,Purple Rain,Purple Rain
HFLuvJXc6SjcJt7d,"['pop', 'rain']",Cher,"Rain, Rain",I Got You Babe / It's Gonna Rain [Digital 45]


In [35]:
getTopValues('0009fFIM1eYThaPg', df_jaccardDistance_word2vec).head(11)

,genre,artist,song,album_name
id,,,,
0009fFIM1eYThaPg,['pop'],Cheryl,Rain on Me,3 Words
ao7dLWmpTymdh4NP,"['pop', 'britpop', 'rain', 'jazz', 'rock', 'electropop', 'soul', 'post rock', 'funk', 'acid jazz', 'alternative pop', 'electronica', 'alternative rock', 'power pop']",Mika,Rain,The Boy Who Knew Too Much
Bp2C0upIsbgDIj6g,"['pop', 'rain', 'singer songwriter', 'water', 'riot grrrl']",Cyndi Lauper,Who Let in the Rain,Hat Full Of Stars
qhUahI7HuwtK1PxU,"['hard rock', 'alternative metal', 'gothic rock', 'industrial rock', 'blues rock']",In This Moment,River of Fire,Ritual
2HMIpSuCforil1f7,['rock nacional'],Biquini Cavadão,Chove Chuva,Agora
AnOdZ5pDIL2LhtCb,"['quiet storm', 'soul', 'rain', 'r b', 'pop', 'hip hop', 'smooth soul', 'funk', 'jazz funk', 'new jack swing', 'rhythm and blues', 'girl group']",SWV,Rain,Release Some Tension
HHB09OecPCe3AVRh,"['pop', 'synthpop', 'brega', 'tecnobrega']",Jaloo,Chuva,#1
PJCZM1so3O1xzQPH,"['pop', 'rock', 'soft rock', 'classic rock', 'pop rock', 'easy listening', 'rain', 'singer songwriter', 'progressive rock', 'soul', 'emo', 'choral', 'gospel', 'jazz rock']",Phil Collins,I Wish It Would Rain Down,...But Seriously (Deluxe Edition)
Nv1MG1CNMFIvY5Zc,"['mpb', 'bossa nova', 'samba', 'samba rock', 'tropicalia', 'candomble', 'classic rock', 'soul', 'funk', 'latin', 'jazz fusion', 'swing']",Jorge Ben Jor,Chove Chuva,Samba Esquema Novo


In [36]:
getTopValues('0009fFIM1eYThaPg', df_innerProductDistance_bert).head(11)

,genre,artist,song,album_name
id,,,,
0009fFIM1eYThaPg,['pop'],Cheryl,Rain on Me,3 Words
wXcMv63aWS4KEPm8,"['pop', 'classic rock', 'country', 'rain', 'rockabilly', 'rock', 'easy listening', 'rock and roll', 'pop rock']",The Everly Brothers,Crying in the Rain,The Golden Hits Of The Everly Brothers
WAFIWIziIPINi0MC,"['pop', 'new wave', 'synthpop', 'rain', 'rock', 'pop rock', 'new romantic', 'easy listening', 'soft rock']",a-ha,Crying in the Rain,East Of The Sun West Of The Moon
2HMIpSuCforil1f7,['rock nacional'],Biquini Cavadão,Chove Chuva,Agora
qhUahI7HuwtK1PxU,"['hard rock', 'alternative metal', 'gothic rock', 'industrial rock', 'blues rock']",In This Moment,River of Fire,Ritual
dClpcYbLcrokcbVm,"['pop', 'indie pop', 'electronica', 'electropop', 'singer songwriter', 'indie rock', 'indietronica', 'soul', 'experimental', 'world', 'art rock', 'freak folk', 'alternative pop']",Lykke Li,Let It Fall,Youth Novels
aY7VhvjZJ0vLZX5F,"['soul', 'rain', 'neo soul', 'singer songwriter', 'r b', 'doo wop']",Mayer Hawthorne,I Wish It Would Rain,A Strange Arrangement
rK32HeW1yxenc3k6,"['reggae', 'rock', 'experimental dub', 'rock nacional']",Sumo,La Gota En El Ojo,After Chabon
xEOnJrFpnOiPhdxm,"['gothic metal', 'alternative metal', 'metal']",Lacuna Coil,One Cold Day,Broken Crown Halo


In [37]:
# Restore col width as default
#pd.reset_option("max_columns")
#pd.reset_option("max_colwidth")

pd.options.display.max_colwidth = 5
pd.options.display.max_columns = 20

In [38]:
print(pd.options.display.max_columns)
print(pd.options.display.max_colwidth)

20
5


In [39]:
# End of program
# Always at the end the program saves the results in to one file, if the file doesn't exist it creates a new one.
df_cosineDistance_tfidf.to_csv('cosine_distances_tfidf.csv',sep=',')
df_innerProductDistance_tfidf.to_csv('innerProduct_distances_tfidf.csv',sep=',')
df_jaccardDistance_tfidf.to_csv('jaccard_distances_tfidf.csv',sep=',')

In [40]:
df_cosineDistance_word2vec.to_csv('cosine_distances_word2vec.csv',sep=',')
df_innerProductDistance_word2vec.to_csv('innerProduct_distances_word2vec.csv',sep=',')
df_jaccardDistance_word2vec.to_csv('jaccard_distances_word2vec.csv',sep=',')

In [41]:
df_cosineDistance_bert.to_csv('cosine_distances_bert.csv',sep=',')
df_innerProductDistance_bert.to_csv('innerProduct_distances_bert.csv',sep=',')
df_jaccardDistance_bert.to_csv('jaccard_distances_bert.csv',sep=',')

# Evaluation of retrieval system

Evaluate your retrieval system using genre as relevance criterion (i.e., a song in the list of retrieved songs (results list) is relevant if any of its genres matches any of the query song's genres). 

## Precision

In [42]:
def get_genres(field):
    return re.findall(r"\'(.*?)\'", field)

In [43]:
# Check if any genre of the song one is in the genres of song two, if yes returns True
def isResultRelevant(songOneGenres, songTwoGenres):
    return any(item in get_genres(songOneGenres) for item in get_genres(songTwoGenres))

### Calculate number of relevant results for each query done
Each column in the dataframes created is a perfomed query

In [44]:
def precision(dfQueries, topNumber):
    
    precision = []
    for query in dfQueries.columns.values:
        querySong = getTopValues(query, dfQueries).loc[query]
        top = getTopValues(query, dfQueries).drop(axis=0, index=[query]).head(topNumber)
        # Get if each of the results are relevant, if yes is True
        relevant_results = [isResultRelevant(querySong['genre'], genres) for genres in top['genre'].values]
        precision.append(np.mean(relevant_results))
        
    return np.mean(precision)

In [45]:
def meanAveragePrecision(dfQueries, topNumber):
    
    AP_ = []
    for query in dfQueries.columns.values: # For each query done
        querySong = getTopValues(query, dfQueries).loc[query] # Data of song queried
        top = getTopValues(query, dfQueries).drop(axis=0, index=[query]).head(topNumber) # Top n songs values
        # Get if each of the results are relevant, if yes is True
        # Array containing for each result if it is relevant or not eg. Top5 [True, True, False, True, False]   
        relevant_results = [isResultRelevant(querySong['genre'], genres) for genres in top['genre'].values]
    
        REL = np.sum(relevant_results)
        # print([relevant_results[i] * (np.sum(relevant_results[:i+1]) / (i+1))   for i in range(topNumber)])
        
        AP = (1/REL) * np.sum([relevant_results[i] * (np.sum(relevant_results[:i+1]) / (i+1))   for i in range(topNumber)])
        
        AP_.append(AP)
        
    return np.mean(AP_)

In [46]:
p10 = meanAveragePrecision(df_innerProductDistance_bert, 10)
p100 = meanAveragePrecision(df_innerProductDistance_bert, 100)
print("MAP@10  :", p10)
print("MAP@100 :", p100)

MAP@10  : 0.6149470899470899
MAP@100 : 0.49664190904709304


In [47]:
p10 = meanAveragePrecision(df_jaccardDistance_word2vec, 10)
p100 = meanAveragePrecision(df_jaccardDistance_word2vec, 100)
print("MAP@10  :", p10)
print("MAP@100 :", p100)

MAP@10  : 0.8039115646258503
MAP@100 : 0.6071114981632889


In [48]:
p10 = meanAveragePrecision(df_cosineDistance_tfidf, 10)
p100 = meanAveragePrecision(df_cosineDistance_tfidf, 100)
print("MAP@10  :", p10)
print("MAP@100 :", p100)

MAP@10  : 0.7498820546737212
MAP@100 : 0.5784505506751908


# MRR

In [49]:
def meanReciprocalRank(dfQueries, topNumber):
    RR = []
    for query in dfQueries.columns.values: # For each query done
        querySong = getTopValues(query, dfQueries).loc[query] # Data of song queried
        top = getTopValues(query, dfQueries).drop(axis=0, index=[query]).head(topNumber) # Top n songs values
        # Get if each of the results are relevant, if yes is True
        # Array containing for each result if it is relevant or not eg. Top5 [True, True, False, True, False]   
        relevant_results = [isResultRelevant(querySong['genre'], genres) for genres in top['genre'].values]
        # print(relevant_results)
     
        min_idx_rel = relevant_results.index(True) + 1
        # print(min_idx_rel)
        RR.append(1/min_idx_rel) 
        
    return np.mean(RR)

In [50]:
mrr10 = meanReciprocalRank(df_innerProductDistance_bert, 10)
mrr100 = meanReciprocalRank(df_innerProductDistance_bert, 100)
print("MRR@10  :", mrr10)
print("MRR@100 :", mrr100)

MRR@10  : 0.6666666666666666
MRR@100 : 0.6666666666666666


In [51]:
mrr10 = meanReciprocalRank(df_jaccardDistance_word2vec, 10)
mrr100 = meanReciprocalRank(df_jaccardDistance_word2vec, 100)
print("MRR@10  :", mrr10)
print("MRR@100 :", mrr100)

MRR@10  : 1.0
MRR@100 : 1.0


In [52]:
mrr10 = meanReciprocalRank(df_cosineDistance_tfidf, 10)
mrr100 = meanReciprocalRank(df_cosineDistance_tfidf, 100)
print("MRR@10  :", mrr10)
print("MRR@100 :", mrr100)

MRR@10  : 1.0
MRR@100 : 1.0


# nDCG

In [53]:
# Gain for the user is considered with the genre, if the song retrieved contains the genre the gain will be 1, 
# if not 0.

# Also the gain could be considered in descending order from k,..., 0

# For example:
# Given the array of results marked as relevant  [1, 0, 0, 1, 1] for @k = @5
# For the first consideration the user gain will be the same d1(1), d2(0), d3(0), d4(1), d5(1)

# Not implemented
# For the second one the user gain could be d1(5), d2(0), d3(0), d4(4), d5(3),
# reducing in one the relevance everytime a new relevant doc appears

def ndcgMean(dfQueries, topNumber):
    ndcg = []
    for query in dfQueries.columns.values: # For each query done
        querySong = getTopValues(query, dfQueries).loc[query] # Data of song queried
        top = getTopValues(query, dfQueries).drop(axis=0, index=[query]).head(topNumber) # Top n songs values
        # Get if each of the results are relevant, if yes is True
        # Array containing for each result if it is relevant or not eg. Top5 [True, True, False, True, False]   
        relevant_results = [isResultRelevant(querySong['genre'], genres) for genres in top['genre'].values]
        sorted_results = sorted(relevant_results, reverse=True)
        # print(relevant_results)
        # print(sorted_results)
        # print(".........")
        dcg = np.sum([ res/np.log2(i+1) if i+1 > 1 else float(res) for i,res in enumerate(relevant_results)])
        # print([ res/np.log2(i+1) if i+1 > 1 else float(res) for i,res in enumerate(relevant_results)])
        idcg = np.sum([ res/np.log2(i+1) if i+1 > 1 else float(res) for i,res in enumerate(sorted_results)])
        ndcg.append(dcg / idcg)
        # print(dcg, idcg)
      
        
    return (ndcg, np.mean(ndcg))

In [54]:
_, ndcg10 = ndcgMean(df_innerProductDistance_bert, 10)
_, ndcg100 = ndcgMean(df_innerProductDistance_bert, 100)
print("NDCG@10  :", ndcg10)
print("NDCG@100 :", ndcg100)

NDCG@10  : 0.783060751802182
NDCG@100 : 0.791442108944539


In [55]:
_, ndcg10 = ndcgMean(df_jaccardDistance_word2vec, 10)
_, ndcg100 = ndcgMean(df_jaccardDistance_word2vec, 100)
print("NDCG@10  :", ndcg10)
print("NDCG@100 :", ndcg100)

NDCG@10  : 0.8804440943269228
NDCG@100 : 0.8583213105391603


In [56]:
_, ndcg10 = ndcgMean(df_cosineDistance_tfidf, 10)
_, ndcg100 = ndcgMean(df_cosineDistance_tfidf, 100)
print("NDCG@10  :", ndcg10)
print("NDCG@100 :", ndcg100)

NDCG@10  : 0.8487915974057271
NDCG@100 : 0.840899373902425
